In [1]:
import numpy as np
import pandas as pd
from utils import dataset_name
print(f"Experiments over {dataset_name}")

Experiments over hepatoCellular


In [2]:
train = pd.read_csv(f"../../datasets/train/{dataset_name}.csv")
test = pd.read_csv(f"../../datasets/test/{dataset_name}.csv")

In [3]:
# delete instances with duration = 0.0
idx = train[train["duration"] == 0].index.values if (train["duration"] == 0).any() else None
if idx is not None:
    train.drop(index=idx, inplace=True)
idx = test[test["duration"] == 0].index if (test["duration"] == 0).any() else None
if idx is not None:
    test.drop(index=idx, inplace=True)

In [4]:
print(f"TRAIN SHAPE {train.shape}, TEST SHAPE {test.shape}")

TRAIN SHAPE (204, 45), TEST SHAPE (23, 45)


In [5]:
X, y = train.drop(["event", "duration"], axis=1), train[["event", "duration"]]
X_test, y_test = test.drop(["event", "duration"], axis=1), test[["event", "duration"]]

In [6]:
X.head(2)

,Age,Gender,HBsAg,Cirrhosis,ALT,AST,AFP,Tumorsize,Tumordifferentiation,Vascularinvasion,...,CD15NR,CD68NR,CD4TR,CD8TR,CD20TR,CD57TR,CD15TR,CD68TR,Ki67,CD34
0,59,1,1,0,1,1,1,2,2,0,...,0.050232,0.444623,0.014289,0.175472,0.014289,0.014289,0.483644,0.298016,141.066438,26008.000000
1,37,1,1,1,2,2,1,2,1,0,...,0.105249,0.529443,0.096607,0.138420,0.079300,0.033657,0.080774,0.571242,303.015295,46160.267327


# Hyperparameters Tuning
### The `grid_params` dict describe all the possible hyperparameter values i chose to test

In [7]:
from pycox.models import CoxPH
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
from sklearn.model_selection import KFold
from pycox.evaluation import EvalSurv
import torch
import torchtuples as tt
import itertools
import os
import pickle
from torch.nn.modules.activation import ReLU, SELU
from lifelines.utils import concordance_index

np.random.seed(42)
_ = torch.manual_seed(42)

In [8]:
%%time

model_name = "deep_surv"

grid_params = {
    # net params
    "dropout": [0.2, 0.1],
    "num_nodes": [[100], [32, 32], [28,28,100,28,28], [32,128,128,100,100], [16,32,64,64]],
    "activation": [ReLU, SELU],
    # fit params
    "batch_size": [256, 512],
    "epochs": [256, 512],
    # optimizer params
    "lr": [1e-2, 1e-3],
    "weight_decay": [1e-2, 1e-3]
    
}

keys, values = zip(*grid_params.items())
experiments = [dict(zip(keys, v)) for v in itertools.product(*values)]
for experiment in experiments:
    Statistics = {'concordance_td':[], 
                  'ibs': [],
                  'c_index': [],
                  'avg_concordance_td':0.5, 
                  'avg_ibs': 0,
                  'avg_c_index': 0.5,
                  'std_concordance_td': 0,
                  'std_ibs': 0,
                  'std_c_index': 0                  
                 }

    cv = KFold(n_splits=5, shuffle=True, random_state=42)
    for i, (train_index, val_index) in enumerate(cv.split(X)):

        # split to train and test 
        X_train, X_val = X.iloc[train_index].copy(), X.iloc[val_index].copy()
        y_train, y_val = y.iloc[train_index].copy(), y.iloc[val_index].copy()

        # preprocess for y
        lower, upper = np.min(y_train["duration"]), np.max(y_train["duration"])
        idx = np.where((y_val["duration"] > lower) & (y_val["duration"] < upper))[0]
        X_val = X_val.iloc[idx].copy()
        y_val = y_val.iloc[idx].copy()
        
        original_y_train = y_train.copy()
        original_y_val = y_val.copy()
        
        # preprocess step
        cols_standardize = ['CXCL17T', 'CXCL17P', 'CXCL17N', 'CD4T',
                               'CD4N', 'CD8T', 'CD8N', 'CD20T', 'CD20N', 'CD57T', 'CD57N', 'CD15T',
                               'CD15N', 'CD68T', 'CD68N', 'CD4NR', 'CD8NR', 'CD20NR', 'CD57NR',
                               'CD15NR', 'CD68NR', 'CD4TR', 'CD8TR', 'CD20TR', 'CD57TR', 'CD15TR',
                               'CD68TR', 'Ki67', 'CD34']
        cols_leave = ["Gender", "HBsAg", "Cirrhosis", "ALT", "AST", "AFP", 
                                          "Tumorsize", "Tumordifferentiation", "Vascularinvasion", 
                                          "Tumormultiplicity", "Capsulation", "TNM", "BCLC"]
        
        standardize = [([col], StandardScaler()) for col in cols_standardize]
        leave = [(col, None) for col in cols_leave]

        standard_scaler = DataFrameMapper(standardize + leave)
        X_train = standard_scaler.fit_transform(X_train).astype('float32')
        y_train = y_train["duration"].values, y_train["event"].values

        X_val = standard_scaler.transform(X_val).astype('float32')
        y_val = y_val["duration"].values, y_val["event"].values
        val = X_val, y_val
        
        # build the network
        in_features = X_train.shape[1]
        num_nodes = experiment["num_nodes"]
        dropout = experiment["dropout"]
        activation = experiment["activation"]
        out_features = 1
        batch_norm = True
        output_bias = False
        
        net = tt.practical.MLPVanilla(in_features, num_nodes, out_features, batch_norm,
                              dropout, activation, output_bias=output_bias)

        optimizer = tt.optim.Adam(lr=experiment["lr"],
                                  weight_decay = experiment["weight_decay"])
        model = CoxPH(net, optimizer)

        # fit params
        batch_size = experiment["batch_size"]
        epochs = experiment["epochs"]
        callbacks = [tt.callbacks.EarlyStopping()]
        verbose = False

        # train the model
        log = model.fit(X_train, y_train, batch_size, epochs, callbacks, verbose,
                val_data=val, val_batch_size=batch_size)


        # model evaluation

        ## pycox measures
        _ = model.compute_baseline_hazards(X_train, y_train)
        estimate_surv = model.predict_surv_df(X_val)
        ev = EvalSurv(estimate_surv, y_val[0], y_val[1], censor_surv='km')

        tmp_times = np.sort(y_train[0])
        times = np.array([tmp_times[i] for i in range(0, len(tmp_times), 10)])

        concordance_td = ev.concordance_td('antolini')
        ibs = ev.integrated_brier_score(times)

        ## lifelines measures
        estimate = np.mean(1-estimate_surv, axis=0)
        c_index = 1 - concordance_index(event_times=original_y_val["duration"], 
                          predicted_scores= estimate, 
                          event_observed=original_y_val["event"])

        # store statistics in Statistics dict
        Statistics["c_index"].append(c_index)
        Statistics["ibs"].append(ibs)
        Statistics["concordance_td"].append(concordance_td)

    # summarise cross validation scores
    Statistics["avg_concordance_td"] = np.mean(Statistics["concordance_td"])
    Statistics["avg_ibs"] = np.mean(Statistics["ibs"])
    Statistics["avg_c_index"] = np.mean(Statistics["c_index"])    
    Statistics["std_concordance_td"] = np.std(Statistics["concordance_td"])
    Statistics["std_ibs"] = np.std(Statistics["ibs"])    
    Statistics["std_c_index"] = np.std(Statistics["c_index"])

    # save the model for later
    try:
        os.mkdir(f"statistics/{model_name}/models")

    except OSError as error: 
        
        file_name = f"{model_name}_"
        for k,v in experiment.items():
            file_name += f"{str(k)}_{str(v)}_"
        
        # dump model to files
        path_sc = os.path.join(f"statistics/{model_name}/models", f"sc_{file_name}.pkl")            
        path_net = os.path.join(f"statistics/{model_name}/models", f"net_{file_name}")
        path_weights = os.path.join(f"statistics/{model_name}/models", f"weights_{file_name}")
        
        with open(path_sc, 'wb') as f:
            pickle.dump(standard_scaler, f)
            
        model.save_net(path_net)
        model.save_model_weights(path_weights)
    
        # dump statistics to pickle    
        file_name += ".pkl"
        path = os.path.join(f"statistics/{model_name}", file_name)

        with open(path, 'wb') as f:
            pickle.dump(Statistics, f)

print("DONE")

DONE
CPU times: user 10min 25s, sys: 13min 38s, total: 24min 4s
Wall time: 9min 37s


# Extract the best model for each one of the measurements
`c_index` `concordance_td`, `integrated brier score`

In [9]:
best_statistics = {'concordance_td': -1, 
                  'ibs': 1,
                   'c_index': -1
                 }
experiments_list = os.listdir(f"statistics/{model_name}")
for experiment in experiments_list:
    if not experiment.startswith(model_name):
        continue
        
    path = os.path.join(f"statistics/{model_name}", experiment)
    stats = pickle.load(open(path, "rb" ))
    df = pd.DataFrame(stats)

    concordance_td = df["avg_concordance_td"][0]
    std_concordance_td = df["std_concordance_td"][0]
    ibs = df["avg_ibs"][0]
    std_ibs = df["std_ibs"][0]    
    c_index = df["avg_c_index"][0]
    std_c_index = df["std_c_index"][0]    
    
    if c_index > best_statistics["c_index"]:
        best_statistics["c_index"] = c_index
        best_statistics["c_index_std"] = std_c_index
        best_statistics["c_index_params"] = experiment
    
    if concordance_td > best_statistics["concordance_td"]:
        best_statistics["concordance_td"] = concordance_td
        best_statistics["concordance_td_std"] = std_concordance_td
        best_statistics["concordance_td_params"] = experiment
        
    if ibs < best_statistics["ibs"]:
        best_statistics["ibs"] = ibs
        best_statistics["ibs_std"] = std_ibs
        best_statistics["ibs_params"] = experiment



path = os.path.join(f"statistics/{model_name}", "best_model.pkl")
with open(path, 'wb') as f:
    pickle.dump(best_statistics, f)    
    
print("DONE")

DONE
